In [1]:
import csv
import tabula
import numpy
import pandas as pd

In [2]:
df_big = pd.read_csv(open(r'C:\Users\ME\Documents\data.csv'))

C:\Users\ME\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df = pd.DataFrame()
df[['game_id', 'home_team', 'away_team', 'posteam_type', 'yardline_100', 'game_seconds_remaining', 'play_type', 'yards_gained', 'total_home_score', 'total_away_score', 'sack']] = df_big[['game_id', 'home_team', 'away_team', 'posteam_type', 'yardline_100', 'game_seconds_remaining', 'play_type', 'yards_gained', 'total_home_score', 'total_away_score', 'sack']]

In [4]:
def median_running(run_df):
    return run_df['yards_gained'].median()

In [5]:
def median_passing(pass_df):
    return pass_df['yards_gained'].median()

In [6]:
def pull(game_id, home, df):
    #Get dataframe for all plays in game
    game_df = df[df['game_id'] == game_id]
    
    #Get dataframe for all run plays where home team has possession
    if(home):
        type = "home"
    else:
        type = "away"
    team_df = game_df[game_df['posteam_type'] == type]
    run_df = team_df[team_df['play_type'] == "run"]
    pass_df = team_df[team_df['play_type'] == "pass"]
    
    median_rush = median_running(run_df)
    median_pass = median_passing(pass_df)
    
    return {'Median Rush Yards' : median_rush, 'Median Pass Yards' : median_pass}

In [7]:
def num_sacks(team_df):   
    return team_df['sack'].sum()

In [8]:
def num_bp(run_df, pass_df):  
    big_runs = run_df[run_df['yards_gained'] > 12]
    big_passes = pass_df[pass_df['yards_gained'] > 16]
    return len(big_runs)+len(big_passes)

In [9]:
def num_rush(run_df):
    return len(run_df)

In [10]:
def time_of_possession(home, game_df):
        
    if(len(game_df) < 2):
        time = 0
    else:
        if(game_df['posteam_type'].iloc[0] == home):
            time_current = game_df['game_seconds_remaining'].iloc[0]
            time_next = game_df['game_seconds_remaining'].iloc[1]
            time = time_current - time_next + time_of_possession(home, game_df[1:])
        else:
            time = time_of_possession(home, game_df[1:]) 
    return time

In [11]:
def average_start(home, game_df):
    yardline = 0
    count = 0
    i = 1
    average_start = 0
    
    if(home):
        type = "home"
    else:
        type = "away"
        
    while(i < len(game_df)):
        punt = game_df['play_type'].iloc[i-1] == "punt"
        kickoff = game_df['play_type'].iloc[i-1] == "kickoff"
        to_home = game_df['posteam_type'].iloc[i] == type
        
        if((punt or kickoff) and to_home):
            yardline += game_df['yardline_100'].iloc[i]
            count += 1
        i += 1
    if(count > 0):
        average_start = 100 - float(yardline / count)
    
    return average_start

In [ ]:
def turnovers_q1(home, game_df):


In [ ]:
def turnovers_q2(home, game_df):
    

In [ ]:
def turnovers_q3(home, game_df):

In [ ]:
def turnovers_q4(home, game_df):

In [ ]:
def fumbles(home, game_df):
    
    #yardline
    return 0;

In [ ]:
def average_completion_streak(home, game_df):
    

In [12]:
def total_points(home, game_df):
    if(home):
        score = game_df['total_home_score'].max()
    else:
        score = game_df['total_away_score'].max()
        
    return score

In [13]:
def summary_row(game_id, home, df):
    
    #Get dataframe for all plays in game
    game_df = df[df['game_id'] == game_id]
    
    #Get dataframe for all run plays where home team has possession
    if(home):
        type = "home"
    else:
        type = "away"
    
    team_df = game_df[game_df['posteam_type'] == type]
    run_df = team_df[team_df['play_type'] == "run"]
    pass_df = team_df[team_df['play_type'] == "pass"]
    kickoff_df = team_df[team_df['play_type'] == "kickoff"]
    punt_df = team_df[team_df['play_type'] == "punt"]        
    
    #Find median rushing and passing yards
    median_rush = median_running(run_df)
    median_pass = median_passing(pass_df)
    
    #Find total number of sacks
    sacks = num_sacks(team_df)
    
    #Find total number of big plays
    big_plays = num_bp(run_df, pass_df)
    
    #Find number of rush
    num_runs = num_rush(run_df)
    
    #Find time of possession
    top = time_of_possession(type, game_df)
        
    #Average starting field position
    starting_fp = average_start(home, game_df)
    
    #Find total points
    score = total_points(home, game_df)
    
    #Create new row to add to summary dataframe
    return {'Median Rush Yards' : median_rush, 'Median Pass Yards' : median_pass, 'Sacks' : sacks, 'Big Plays' : big_plays, 'Number Of Runs' : num_runs, 'Time Of Possession' : top, 'Average Starting Field Position' : starting_fp, 'Total Points' : score}
    #Add row
    #df_summary.append(new_row, ignore_index = True)

In [14]:
def make_game_df(df):
    game_df = pd.DataFrame()
    df_away = pd.DataFrame()
        
    game_df[['Team', 'Game_ID']] = df[['home_team', 'game_id']]
    game_df['Home'] = True
    df_away[['Team', 'Game_ID']] = df[['away_team', 'game_id']]
    df_away['Home'] = False
    
    game_df = game_df.append(df_away, ignore_index=True)
    game_df_noduplicates = game_df.drop_duplicates(ignore_index=True)
    
    return game_df_noduplicates

In [15]:
games_df = make_game_df(df)
games_df.head

<bound method NDFrame.head of      Team     Game_ID   Home
0     PIT  2009091000   True
1     CLE  2009091304   True
2      NO  2009091307   True
3      TB  2009091308   True
4     HOU  2009091305   True
...   ...         ...    ...
5047  DAL  2018121605  False
5048  SEA  2018121609  False
5049   NE  2018121610  False
5050  PHI  2018121611  False
5051   NO  2018121700  False

[5052 rows x 3 columns]>

In [16]:
summary_row(2009091000, False, df)

{'Median Rush Yards': 2.0,
 'Median Pass Yards': 3.5,
 'Sacks': 1.0,
 'Big Plays': 4,
 'Number Of Runs': 25,
 'Time Of Possession': 1693.0,
 'Average Starting Field Position': 18.599999999999994,
 'Total Points': 10}

In [17]:
games_df = make_game_df(df)

df_summary1 = pd.DataFrame()

df_summary1['Team'] = games_df['Team']
df_summary1['Game_ID'] = games_df['Game_ID']
df_summary1['Home'] = games_df['Home']

df_summary2 = pd.DataFrame()

for i in range(len(games_df)):
        row = games_df.iloc[i]
        df_summary2 = df_summary2.append(summary_row(row[1], row[2], df), ignore_index=True)

df_summary = pd.concat([df_summary1, df_summary2], axis=1)


In [18]:
df_summary.to_csv(r'C:\Users\ME\Documents\summary.csv', index=False)